In [3]:
import timeit
import cv2 as cv
import numpy as np

imgArr = cv.imread("./Images/LowContrast.tif", cv.IMREAD_GRAYSCALE)


In [68]:
def histogram_arr(arr, numLevels=256):
    histogram = np.zeros(numLevels)
    for val in np.nditer(arr): #Consider not using nditer for performance
        histogram[val] += 1
    return histogram

def histogram_arr2(arr, numLevels=256):
    histogram = np.zeros(numLevels)
    for row in arr:
        for val in row:
            histogram[val] += 1
    return histogram

def hist_eq(imgArr, numLevels=256):
    numPixels = imgArr.shape[0]*imgArr.shape[1]
    cdf = np.zeros(numLevels)
    histogram = histogram_arr(imgArr)
    # Get cumulative distribution function
    for i in range(numLevels):
        sum = 0
        for j in range(i+1):
            sum += histogram[j]
        cdf[i] = sum/numPixels 
    # Apply cdf as transformation function, scaled by number of levels
    outputArr = np.zeros(imgArr.shape, dtype=int)
    for (ind, val) in np.ndenumerate(imgArr):
        outputArr[ind] = (numLevels-1)*cdf[val]
    return outputArr

def hist_eq2(imgArr, numLevels=256):
    numPixels = imgArr.shape[0]*imgArr.shape[1]
    cdf = np.zeros(numLevels)
    histogram = histogram_arr2(imgArr)
    # Get cumulative distribution function
    for i in range(numLevels):
        sum = 0
        for j in range(i+1):
            sum += histogram[j]
        cdf[i] = sum/numPixels 
    # Apply cdf as transformation function, scaled by number of levels
    outputArr = np.zeros(imgArr.shape, dtype=int)
    for (ind, val) in np.ndenumerate(imgArr):
        outputArr[ind] = (numLevels-1)*cdf[val]
    return outputArr


startTime = timeit.default_timer()
output = hist_eq(imgArr)
totalTime = timeit.default_timer() - startTime 

print(totalTime)

0.18495149991940707


In [70]:

def time_function(func, numIterations=1, *args, **kwargs):
    timeSum = 0
    for i in range(numIterations):
        startTime = timeit.default_timer()
        func(*args, **kwargs)
        timeSum += timeit.default_timer() - startTime 
    avgTime = timeSum/numIterations
    return avgTime

print(time_function(hist_eq, 200, imgArr))
print(time_function(hist_eq2, 200, imgArr))


0.17745992500102148
0.1700893725012429
